In [0]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image  , ImageDraw 
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import os

In [0]:
torch.__version__

'1.5.0+cu101'

In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd drive/My Drive/Resnet_Beta-Version
!ls

/content/drive/My Drive/Resnet_Beta-Version
'alpha-Resnet(Raw+Face+Hand).ipynb'
'alpha-Resnet(Raw+Face).ipynb'
'alpha-Resnet(Raw+Hand).ipynb'
'alpha-Resnet(Raw).ipynb'
'Arrange Dataset.ipynb'
'beta-Resnet(Raw+Face+Hand).ipynb'
'beta-Resnet(Raw+Face+Hand+Pose)-Pose as Depth.ipynb'
'beta-Resnet(Raw+Face).ipynb'
'beta-Resnet(Raw+Hand).ipynb'
'beta-Resnet(Raw).ipynb'
'cropped_Resnet(Raw).ipynb'
 dataset
 dump
 epoch_65.pth
 extract_faceandhand_jorehgaye.ipynb
 fallingDemo.ipynb
 fallingDemo_modified_2.py
 fallingDemo_modified.py
 fallingDemo.py
 loss
 out.jpg
 putTXT_rightplace.ipynb
'Testbeta-Resnet(Raw+Face+Hand).ipynb'
'Testbeta-Resnet(Raw+Face+Hand+Pose)-Pose as Depth.ipynb'
'testbeta-Resnet(Raw+Face).ipynb'
'Testbeta-Resnet(Raw+Hand).ipynb'
'Testbeta-Resnet(Raw).ipynb'
 TestingModels_first.ipynb
 TestingModels_second.ipynb
 weights


In [0]:

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
}

In [0]:
class MyImageFolder(datasets.ImageFolder):
    
    def __init__(self, root, transform=None):
        super(MyImageFolder,self).__init__(root)

        self.transform = transform

    def __getitem__(self, index):

        #imgs = self.imgs[index]
        #print(imgs)

        path, target = self.samples[index]
        #sample = self.loader(path)
        #print("Sample: ",sample)
        

        hm_transform = transforms.Compose([
                          transforms.Resize((224,224)),
                          transforms.ToTensor()

                                ])

        #print(path)
        #print(type(path))
        #all_images = []
        #all_labels = []
        #all_hm_face = []
        #all_hm_hand = []
        # target_acquired - 0: image path -- 1: label
        #for i in len(path):

        img_path = path
        #print("--i: %d , TA: %s"%(i,target_acquired))

        
        #img_path = target_acquired
        #label =  target_acquired[1]

        #print(img_path)

        image = Image.open(img_path)
        img_w , img_h = image.size

    
        filename = os.path.basename(img_path)
        ann_path = img_path.replace(filename, filename.split('.')[0] + '.txt')
        hm_face , hm_hand = self.get_heatmap(ann_path,img_w,img_h)  

        #image = np.dstack((image, hm_face))

        
        # Transforms
        if self.transform is not None:
            
            image = self.transform(image)
            hm_face = hm_transform(hm_face)

        image = torch.cat((image,hm_face))
        #if self.target_transform is not None:
        #    target = self.target_transform(target)
        
        
        # For Face
            #image = np.dstack((image, hm_face))

            #all_images.append(img_path)
            #all_labels.append(label)
            #all_hm_face.append(hm_face)
            #all_hm_hand.append(hm_hand)

        #return path, all_hm_face, all_hm_hand, target
        
        return image, target

    def get_heatmap(self,ann_path, img_w , img_h):
        
        hm_0 = Image.new('L', (img_w, img_h))
        hm_1 = Image.new('L', (img_w, img_h))
        
        try:

            with open(ann_path) as f:

                for line in f.readlines():

                    if line == '\n':
                        continue

                    line = line.split(' ')

                    cx = float(line[1])
                    cy = float(line[2])
                    w = float(line[3])
                    h = float(line[4])

                    xmin =  (cx - (w/2)) * img_w
                    ymin = (cy - (h/2)) * img_h
                    xmax = (cx + (w/2)) * img_w
                    ymax = (cy + (h/2)) * img_h

                    if line[0] == '0':
                        hmDraw = ImageDraw.Draw(hm_0)   
                    else:
                        hmDraw = ImageDraw.Draw(hm_1)   
                    
                    hmDraw.rectangle([xmin,ymin,xmax,ymax], fill ="#ffffff")
        except FileNotFoundError:
            print("FileNotFoundError: %s"%ann_path)
            #pass

        return hm_0, hm_1
"""
Test Code


path, target = self.samples[index]
sample = self.loader(path)
if self.transform is not None:
  sample = self.transform(sample)
if self.target_transform is not None:
  target = self.target_transform(target)
# this is where you load your resnet data
resnet_path = os.path.join(os.path.splitext(path)[0], '.pth')  # replace image extension with .pth
resnet = torch.load(resnet_path)  # load the stored features
return sample, resnet, target
"""
#dataset = MyImageFolder("falling_dataset/val/")
#re, labels = dataset[1:3]
#print(re)
#print(labels)
#re[2].save('pred_face.jpeg')
#re[3].save('pred_hand.jpeg')

"\nTest Code\n\n\npath, target = self.samples[index]\nsample = self.loader(path)\nif self.transform is not None:\n  sample = self.transform(sample)\nif self.target_transform is not None:\n  target = self.target_transform(target)\n# this is where you load your resnet data\nresnet_path = os.path.join(os.path.splitext(path)[0], '.pth')  # replace image extension with .pth\nresnet = torch.load(resnet_path)  # load the stored features\nreturn sample, resnet, target\n"

In [0]:
input_path = './dataset/alpha/Fall'

image_datasets = {
    'train': 
    MyImageFolder(input_path + '/train', data_transforms['train']),
     'validation': 
    MyImageFolder(input_path + '/val', data_transforms['validation'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=64,
                                shuffle=True,
                                num_workers=2),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=32,
                                shuffle=True,
                                num_workers=2)  # for Kaggle
}

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [0]:
model = models.resnet50(pretrained=True).to(device)

model.cuda()
#print(model)

depth = 4


for param in model.parameters():
    param.requires_grad = False   


#------------ Modify First Layer
model.conv1 = nn.Conv2d(depth, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False).to(device)
model.conv1.requires_grad = True

#------------ Modify Last Layer
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2),
               nn.Sigmoid()).to(device)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(),lr=0.001)

In [0]:
def train_model(model, criterion, optimizer, resume_epoch= 0 , num_epochs=3):
    new_write = True
    for epoch in range(num_epochs):
        
    
        print('Epoch {}/{}'.format(resume_epoch + epoch+1, num_epochs))
        print('-' * 10)
        

        for phase in ['train','validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            count = 0
            
            with torch.set_grad_enabled(phase == 'train'):
              for inputs, labels in dataloaders[phase]:
                  inputs = inputs.to(device)
                  labels = labels.to(device)

                  outputs = model(inputs)
                  loss = criterion(outputs, labels)

                  if phase == 'train':
                      
                      optimizer.zero_grad()
                      loss.backward()
                      optimizer.step()

                  _, preds = torch.max(outputs, 1)
                  running_loss += loss.item() * inputs.size(0)
                  running_corrects += torch.sum(preds == labels.data)

                  count = count+1                
                  if (count%40==0):
                    print (count)
             

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            if (os.path.isfile('loss/alpha/Fall/raw_face.txt')):
              new_write=False  
            else :
              print ("No file")
            f = open('loss/alpha/Fall/raw_face.txt','a+')
            if  (new_write == True ):
              print ("here")
              f.write('{} loss: {:.4f}, acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            else :
              f.write ('\n')
              f.write('{} loss: {:.4f}, acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            
        
        torch.save(model, 'weights/alpha/Fall/raw-face/epoch_%d.pth'%(resume_epoch + epoch + 1))
        f.close()


    return model

In [19]:
!pwd

epoch = 87
model = (torch.load('weights/alpha/Fall/raw-face/epoch_%d.pth'%(epoch)))
model.cuda()

model_trained = train_model(model, criterion, optimizer, epoch ,num_epochs=100-epoch)


/content/drive/My Drive/Resnet_Beta-Version
Epoch 88/13
----------
FileNotFoundError: ./dataset/alpha/Fall/train/Falling/VID20191202161842175.txt
FileNotFoundError: ./dataset/alpha/Fall/train/NonFalling/VID2019120216262966.txt
FileNotFoundError: ./dataset/alpha/Fall/train/NonFalling/VID20191202162629137.txt
FileNotFoundError: ./dataset/alpha/Fall/train/NonFalling/VID20191202162629544.txt
FileNotFoundError: ./dataset/alpha/Fall/train/NonFalling/VID20191202162629520.txt
FileNotFoundError: ./dataset/alpha/Fall/train/Falling/17304.txt
FileNotFoundError: ./dataset/alpha/Fall/train/Falling/IMG_5790_Frame76.txt
FileNotFoundError: ./dataset/alpha/Fall/train/Falling/VID20191202161842172.txt
FileNotFoundError: ./dataset/alpha/Fall/train/NonFalling/VID20191202162629516.txt
FileNotFoundError: ./dataset/alpha/Fall/train/NonFalling/VID20191202162629408.txt
FileNotFoundError: ./dataset/alpha/Fall/train/NonFalling/VID20191202162629543.txt
FileNotFoundError: ./dataset/alpha/Fall/train/NonFalling/VID201

In [0]:
torch.save(model_trained.state_dict(), 'weights/weights_Face_state_dict.h5')
#torch.save(model_trained.state_dict(), 'weights/pytorch/weights_Face_state_dict.h5')
torch.save(model_trained, 'weights/weights_Face.h5')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist

In [0]:
# model = models.resnet50(pretrained=False).to(device)
# model.fc = nn.Sequential(
#                nn.Linear(2048, 128),
#                nn.ReLU(inplace=True),
#                nn.Linear(128, 2)).to(device)
device = 'cpu'
model = torch.load('weights/weights_Face.h5').to(device)
model.eval()


ResNet(
  (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [0]:
!ls


 Annotated			'Resnet(Raw+Face+Hand).ipynb'
'dummy_Resnet(Raw+Face).ipynb'	'Resnet(Raw+Face).ipynb'
 falling_dataset		'Resnet(Raw+Hand).ipynb'
 len_data.ipynb			'Resnet(Raw).ipynb'
 newResNet.ipynb		 txt_files
 Resnet2.0.ipynb		 weights


In [0]:
%cd Resnet
!ls

/content/drive/My Drive/Resnet
 Annotated			'Resnet(Raw+Face+Hand).ipynb'
'dummy_Resnet(Raw+Face).ipynb'	'Resnet(Raw+Face).ipynb'
 falling_dataset		'Resnet(Raw+Hand).ipynb'
 len_data.ipynb			'Resnet(Raw).ipynb'
 newResNet.ipynb		 txt_files
 Resnet2.0.ipynb		 weights


In [0]:
import glob
import random
pathFalling = './falling_dataset/val/Falling/'
pathNonFalling = './falling_dataset/val/NonFalling/'
accurateFalling = 0
accurateNonFalling = 0

list1= [f for f in glob.glob(pathFalling + "*.jpg", recursive=True)]
list2= [f for f in glob.glob(pathNonFalling + "*.jpg", recursive=True)]
print (len(list1))
print (len(list2))
validation_img_pathsFalling=random.sample(list1,len(list1))
validation_img_pathsNonFalling = random.sample(list2,len(list2))
img_listFalling = [Image.open('./' + img_path) for img_path in validation_img_pathsFalling]
img_listNonFalling = [Image.open('./' + img_path) for img_path in validation_img_pathsNonFalling]
print (len(img_listFalling))
print (len(img_listNonFalling))

250
250
250
250


In [0]:
validation_batchFalling = torch.stack([data_transforms['validation'](img).to(device)
                                for img in img_listFalling])
validation_batchNonFalling = torch.stack([data_transforms['validation'](img).to(device)
                                for img in img_listNonFalling])

In [0]:
predictions = []
target = []

iteration = 0

for inputs , labels in dataloaders['validation']:

    if iteration == 150:
      break
    iteration += 1
    inputs = inputs.to(device)
    labels = labels.to(device)

    pred_logits_tensorFalling = model(inputs)
    predictions.append(pred_logits_tensorFalling)
    target.append(labels)

    print("Done")

#pred_logits_tensorFalling
#pred_logits_tensorNonFalling = model(validation_batchNonFalling)
#pred_logits_tensorNonFalling

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


RuntimeError: ignored

In [0]:
out = []

for pred in predictions:

    out.append( torch.argmax(pred).item()) 

acc = 0
for i,tar in enumerate(target):
    if tar.item() == out[i]:
      acc += 1
acc

95

In [0]:
acc /len(out)

0.7661290322580645

In [0]:
len(out)

124

In [0]:
pred_probsFalling = F.softmax(pred_logits_tensorFalling, dim=1).cpu().data.numpy()
print (pred_probsFalling)
pred_probsNonFalling = F.softmax(pred_logits_tensorNonFalling, dim=1).cpu().data.numpy()
print (pred_probsNonFalling)

In [0]:
i=0
accurateFalling=0
for i in range(len(pred_probsFalling)):
  if (pred_probsFalling[i][0]> 0.5):
    accurateFalling= accurateFalling +1 
i=0
accurateNonFalling=0
for i in range(len(pred_probsNonFalling)):
  if (pred_probsNonFalling[i][1]> 0.5):
    accurateNonFalling= accurateNonFalling +1 


In [0]:
overallAccuracy = (accurateNonFalling+accurateFalling)/(len(img_listFalling)+len(img_listNonFalling))

In [0]:
print (accurateFalling)
print (accurateNonFalling)

In [0]:
print (overallAccuracy)

In [0]:
ig, axs = plt.subplots(1, len(img_listFalling), figsize=(150, 150))
for i, img in enumerate(img_listFalling):
    ax = axs[i]
    ax.axis('off')
    ax.set_title("{:.0f}%Falling , {:.0f}% Non-Falling".format(100*pred_probsFalling[i,0],
                                                          100*pred_probsFalling[i,1]))
    ax.imshow(img)

In [0]:
ig, axs = plt.subplots(1, len(img_listNonFalling), figsize=(150, 150))
for i, img in enumerate(img_listNonFalling):
    ax = axs[i]
    ax.axis('off')
    ax.set_title("{:.0f}%Falling , {:.0f}% Non-Falling".format(100*pred_probsNonFalling[i,0],
                                                          100*pred_probsNonFalling[i,1]))
    ax.imshow(img)